# 🏥 Complete Medical Imaging Data Cleaning Pipeline

================================================================================

This notebook performs a **comprehensive data cleaning pipeline** for a medical imaging dataset.

## Pipeline Overview

### **PHASE 1: Initial Cleaning** (Creates clean_entry.csv & flagged.csv)
- Download and load all CSV files from Google Drive
- Clean data_entry.csv (primary source of truth)
- Clean bounding box tables (bbox_2.csv, corrupted bbox)
- Salvage data from corrupted data_entry
- Combine and save initial cleaned data

### **PHASE 2: Full Cleaning** (Creates final_clean.csv)
- **STAGE 1:** Load clean_entry.csv and flagged.csv
- **STAGE 2:** Analyze flagged data quality issues
- **STAGE 3:** Clean flagged data → cleaned2.0.csv
- **STAGE 4:** Extract completely clean data → cleaned3.0.csv  
- **STAGE 5:** Merge all clean data → final_clean.csv

## Output Files
| File | Description |
|------|-------------|
| `clean_entry.csv` | Initially cleaned data with bounding boxes |
| `flagged.csv` | Data with quality issues for review |
| `cleaned2.0.csv` | Flagged data with standardized values |
| `cleaned3.0.csv` | Completely clean flagged data (no NaN) |
| `final_clean.csv` | **Final merged dataset ready for ML** |
| `cleaning_report.txt` | Detailed statistics and summary |

================================================================================

---
# 📦 SECTION 1: Setup and Install Dependencies
---
Install required packages and import all necessary libraries.

In [ ]:
# Install required packages
!pip install gdown -q

# Import all required libraries
import pandas as pd
import numpy as np
import gdown
import os
import re
import shutil
from collections import defaultdict
from datetime import datetime

print("✅ All packages imported successfully!")
print(f"📅 Pipeline started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

---
# 📁 SECTION 2: Mount Google Drive and Create Directories
---
Mount Google Drive for data access and create output directories.

In [ ]:
# Mount Google Drive
from google.colab import drive
print("🔗 Mounting Google Drive...")
drive.mount('/content/drive')

# Create output directories
os.makedirs('/content/cleaned_data', exist_ok=True)
os.makedirs('/content/flagged_data', exist_ok=True)
os.makedirs('/content/logs', exist_ok=True)

print("✅ Google Drive mounted!")
print("📂 Created directories:")
print("   - /content/cleaned_data")
print("   - /content/flagged_data")
print("   - /content/logs")

---
# 📥 SECTION 3: Download CSV Files from Google Drive
---
Define download functions and download all 5 CSV files needed for the pipeline.

In [ ]:
# CSV file links from Google Drive
csv_links = [
    ("https://drive.google.com/file/d/1H6UARor5aXx4_tHACWxE3d4sBY5d3pJF/view?usp=drive_link", "bbox_2.csv"),
    ("https://drive.google.com/file/d/15xqxut8ri1bGc8nMJi7dxv89Xf9QU3BO/view?usp=drive_link", "data_entry_.csv"),
    ("https://drive.google.com/file/d/1KwggxzXcv7DQSH4JN4cTsXooCU1Rehz0/view?usp=drive_link", "data_entry.csv"),
    ("https://drive.google.com/file/d/15RZxzpxgK-V8jYMWb8kzlfJm012KbOxP/view?usp=drive_link", "HACKATHON_CORRUPTED_bbox_list.csv"),
    ("https://drive.google.com/file/d/1Q32l5pH-Fu0JSe8EbVCDIe5xES2Fmo1s/view?usp=drive_link", "HACKATHON_CORRUPTED_data_entry.csv"),
]

def extract_file_id(link):
    """Extract Google Drive file ID from shareable link"""
    if '/file/d/' in link:
        return link.split('/file/d/')[1].split('/')[0]
    return link

def download_file(link, output_name):
    """Download a file from Google Drive"""
    file_id = extract_file_id(link)
    download_url = f'https://drive.google.com/uc?id={file_id}'
    print(f"⬇️  Downloading {output_name}...")
    gdown.download(download_url, output_name, quiet=False)
    print(f"✅ Downloaded {output_name}")

# Download all CSV files
print("=" * 60)
print("📥 DOWNLOADING CSV FILES")
print("=" * 60)

for link, filename in csv_links:
    try:
        download_file(link, f'/content/{filename}')
    except Exception as e:
        print(f"❌ Error downloading {filename}: {e}")

print("\n✅ All downloads complete!")

---
# 📊 SECTION 4: Load All CSV Files
---
Load all downloaded CSV files into pandas DataFrames and inspect their structure.

In [ ]:
print("=" * 60)
print("📊 LOADING CSV FILES")
print("=" * 60)

# Load main data_entry.csv
data_entry = pd.read_csv('/content/data_entry.csv')
print(f"✅ Loaded data_entry.csv: {data_entry.shape}")
print(f"   Columns: {list(data_entry.columns)}")

# Load small data_entry_.csv (optional)
try:
    data_entry_small = pd.read_csv('/content/data_entry_.csv')
    print(f"✅ Loaded data_entry_.csv (small): {data_entry_small.shape}")
except:
    data_entry_small = None
    print("⚠️  data_entry_.csv not available")

# Load bounding box data
bbox_2 = pd.read_csv('/content/bbox_2.csv')
print(f"✅ Loaded bbox_2.csv: {bbox_2.shape}")
print(f"   Columns: {list(bbox_2.columns)}")

# Load corrupted bounding box data
bbox_corrupted = pd.read_csv('/content/HACKATHON_CORRUPTED_bbox_list.csv')
print(f"✅ Loaded HACKATHON_CORRUPTED_bbox_list.csv: {bbox_corrupted.shape}")
print(f"   Columns: {list(bbox_corrupted.columns)}")

# Load corrupted data_entry
data_entry_corrupted = pd.read_csv('/content/HACKATHON_CORRUPTED_data_entry.csv')
print(f"✅ Loaded HACKATHON_CORRUPTED_data_entry.csv: {data_entry_corrupted.shape}")
print(f"   Columns: {list(data_entry_corrupted.columns)}")

print("\n✅ All CSV files loaded successfully!")

---
# 🔧 SECTION 5: Normalize Column Names
---
Standardize column names across different files to ensure consistency.

In [ ]:
def normalize_columns(df):
    """
    Standardize column names across different files.
    Maps variations like 'patient age', 'Patient Age', 'PatientAge' to standard format.
    """
    col_map = {
        # Image Index variations
        'image index': 'Image Index',
        'imageindex': 'Image Index',
        'image_index': 'Image Index',
        
        # Patient Age variations
        'patient age': 'Patient Age',
        'patientage': 'Patient Age',
        'patient_age': 'Patient Age',
        'age': 'Patient Age',
        
        # Patient Gender variations
        'patient gender': 'Patient Gender',
        'patientgender': 'Patient Gender',
        'patient_gender': 'Patient Gender',
        'gender': 'Patient Gender',
        
        # Finding Labels variations
        'finding labels': 'Finding Labels',
        'findinglabels': 'Finding Labels',
        'finding_labels': 'Finding Labels',
        'labels': 'Finding Labels',
        
        # Finding Label (singular - for bbox)
        'finding label': 'Finding Label',
        'findinglabel': 'Finding Label',
        'finding_label': 'Finding Label',
        'label': 'Finding Label',
        
        # Image dimensions
        'originalimage[width': 'OriginalImage[Width',
        'originalimage[height': 'OriginalImage[Height',
        'originalimagewidth': 'OriginalImage[Width',
        'originalimageheight': 'OriginalImage[Height',
        'width': 'OriginalImage[Width',
        'height': 'OriginalImage[Height',
        
        # Bbox coordinates
        'bbox_x': 'x',
        'bbox_y': 'y',
        'bbox_w': 'w',
        'bbox_h': 'h',
        'bboxx': 'x',
        'bboxy': 'y',
        'bboxw': 'w',
        'bboxh': 'h',
    }
    
    # Create lowercase mapping
    df.columns = df.columns.str.strip()
    new_cols = {}
    for col in df.columns:
        col_lower = col.lower().strip()
        if col_lower in col_map:
            new_cols[col] = col_map[col_lower]
        else:
            new_cols[col] = col
    
    df = df.rename(columns=new_cols)
    return df

# Apply normalization to all dataframes
print("🔧 Normalizing column names...")
data_entry = normalize_columns(data_entry)
bbox_2 = normalize_columns(bbox_2)
bbox_corrupted = normalize_columns(bbox_corrupted)
data_entry_corrupted = normalize_columns(data_entry_corrupted)

print("✅ Column normalization complete!")
print(f"\n📋 data_entry columns: {list(data_entry.columns)}")
print(f"📋 bbox_2 columns: {list(bbox_2.columns)}")

---
# 🧹 SECTION 6: Clean Primary Data Entry (data_entry.csv)
---
Clean the main data_entry.csv file - the primary source of truth.

**Cleaning steps:**
1. Remove missing/fake images
2. Clean Patient Age (convert formats, validate range)
3. Validate image dimensions
4. Validate Finding Labels (not empty)
5. Validate Gender (M/F only)

In [ ]:
def clean_data_entry_primary(df):
    """
    Clean the main data_entry.csv - PRIMARY SOURCE OF TRUTH
    
    Checks performed:
    1. Missing/Fake images (patterns: missing, mock, fake)
    2. Invalid age formats (058Y → 58, ?? → NaN)
    3. Unrealistic ages (< 0 or > 120)
    4. Invalid image dimensions
    5. Empty finding labels
    6. Invalid gender values
    
    Returns:
        tuple: (clean_df, flagged_df)
    """
    df = df.copy()
    initial_count = len(df)
    flagged_rows = []
    
    print(f"\n🔍 Starting with {initial_count:,} rows")
    
    # Issue 1: Missing/Fake images
    print("\n1️⃣ Checking for missing/fake images...")
    missing_patterns = ['missing', 'Missing', 'mock', 'fake', 'Missing_file']
    mask_missing = df['Image Index'].str.contains('|'.join(missing_patterns), case=False, na=False)
    flagged = df[mask_missing].copy()
    flagged['flag_reason'] = 'missing_or_fake_image'
    flagged_rows.append(flagged)
    df = df[~mask_missing]
    print(f"   🚩 Flagged: {mask_missing.sum():,} missing/fake images")
    
    # Issue 2: Clean Patient Age
    print("\n2️⃣ Cleaning Patient Age...")
    if 'Patient Age' in df.columns:
        # Convert age format (058Y → 58, ?? → NaN)
        df['Patient Age'] = df['Patient Age'].astype(str)
        df['Patient Age'] = df['Patient Age'].str.replace('Y', '').str.replace('?', '').str.strip()
        
        # Flag invalid ages before conversion
        mask_invalid_age = ~df['Patient Age'].str.match(r'^\d+$', na=False)
        flagged = df[mask_invalid_age].copy()
        flagged['flag_reason'] = 'invalid_age_format'
        flagged_rows.append(flagged)
        
        # Convert to numeric
        df['Patient Age'] = pd.to_numeric(df['Patient Age'], errors='coerce')
        
        # Flag unrealistic ages
        mask_unrealistic = (df['Patient Age'] < 0) | (df['Patient Age'] > 120) | df['Patient Age'].isna()
        flagged = df[mask_unrealistic & ~mask_invalid_age].copy()
        flagged['flag_reason'] = 'unrealistic_age'
        flagged_rows.append(flagged)
        
        df = df[~mask_unrealistic]
        print(f"   🚩 Flagged: {(mask_invalid_age | mask_unrealistic).sum():,} invalid ages")
    
    # Issue 3: Validate image dimensions
    print("\n3️⃣ Validating image dimensions...")
    if 'OriginalImage[Width' in df.columns and 'OriginalImage[Height' in df.columns:
        mask_invalid_dims = (
            (df['OriginalImage[Width'] <= 0) | 
            (df['OriginalImage[Height'] <= 0) |
            df['OriginalImage[Width'].isna() |
            df['OriginalImage[Height'].isna()
        )
        flagged = df[mask_invalid_dims].copy()
        flagged['flag_reason'] = 'invalid_dimensions'
        flagged_rows.append(flagged)
        df = df[~mask_invalid_dims]
        print(f"   🚩 Flagged: {mask_invalid_dims.sum():,} invalid dimensions")
    
    # Issue 4: Validate Finding Labels (not empty)
    print("\n4️⃣ Validating Finding Labels...")
    if 'Finding Labels' in df.columns:
        mask_no_labels = df['Finding Labels'].isna() | (df['Finding Labels'].str.strip() == '')
        flagged = df[mask_no_labels].copy()
        flagged['flag_reason'] = 'no_finding_labels'
        flagged_rows.append(flagged)
        df = df[~mask_no_labels]
        print(f"   🚩 Flagged: {mask_no_labels.sum():,} rows with no labels")
        
        # Clean up labels
        df['Finding Labels'] = df['Finding Labels'].str.strip()
    
    # Issue 5: Validate Gender
    print("\n5️⃣ Validating Gender...")
    if 'Patient Gender' in df.columns:
        mask_invalid_gender = ~df['Patient Gender'].isin(['M', 'F'])
        flagged = df[mask_invalid_gender].copy()
        flagged['flag_reason'] = 'invalid_gender'
        flagged_rows.append(flagged)
        df = df[~mask_invalid_gender]
        print(f"   🚩 Flagged: {mask_invalid_gender.sum():,} invalid gender values")
    
    # Combine all flagged rows
    all_flagged = pd.concat(flagged_rows, ignore_index=True) if flagged_rows else pd.DataFrame()
    
    print(f"\n" + "=" * 50)
    print(f"✅ CLEAN: {len(df):,} rows ({len(df)/initial_count*100:.1f}%)")
    print(f"🚩 FLAGGED: {len(all_flagged):,} rows ({len(all_flagged)/initial_count*100:.1f}%)")
    print("=" * 50)
    
    return df, all_flagged

# Run the cleaning function
print("=" * 70)
print("🧹 PART A: CLEANING DATA_ENTRY (PRIMARY SOURCE OF TRUTH)")
print("=" * 70)

data_entry_clean, data_entry_flagged = clean_data_entry_primary(data_entry)

print(f"\n📊 Results:")
print(f"   Clean records: {len(data_entry_clean):,}")
print(f"   Flagged records: {len(data_entry_flagged):,}")

---
# 📦 SECTION 7: Clean Bounding Box Tables
---
Clean bounding box data from bbox_2.csv and corrupted bbox files.

**Validation checks:**
1. Missing/fake images
2. Invalid box dimensions (9999, -1, negative)
3. Invalid coordinates (negative x, y)
4. Boxes outside image bounds
5. Finding label mismatches

In [ ]:
def clean_bbox_table(bbox_df, valid_images_df, source_name):
    """
    Clean bounding box data
    
    Args:
        bbox_df: Bounding box DataFrame
        valid_images_df: Valid images DataFrame for reference
        source_name: Name of the source file for logging
        
    Returns:
        tuple: (clean_df, flagged_df)
    """
    df = bbox_df.copy()
    initial_count = len(df)
    flagged_rows = []
    
    print(f"\n📦 {source_name}: Starting with {initial_count:,} boxes")
    
    # Check if required columns exist
    required_cols = ['Image Index']
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        print(f"⚠️  Missing columns: {missing_cols}")
        print(f"   Available columns: {list(df.columns)}")
        return pd.DataFrame(), df.copy()
    
    # Check what dimension columns we have
    has_bbox_coords = all(col in df.columns for col in ['x', 'y', 'w', 'h'])
    
    # Create image metadata lookup
    meta_cols = ['Image Index']
    if 'OriginalImage[Width' in valid_images_df.columns:
        meta_cols.append('OriginalImage[Width')
    if 'OriginalImage[Height' in valid_images_df.columns:
        meta_cols.append('OriginalImage[Height')
    if 'Finding Labels' in valid_images_df.columns:
        meta_cols.append('Finding Labels')
    
    image_meta = valid_images_df[meta_cols].set_index('Image Index').to_dict('index')
    
    # Issue 1: Missing/Fake images
    print(f"1️⃣ Checking for missing/fake images...")
    missing_patterns = ['missing', 'Missing', 'mock', 'fake']
    mask_missing = df['Image Index'].str.contains('|'.join(missing_patterns), case=False, na=False)
    flagged = df[mask_missing].copy()
    flagged['flag_reason'] = f'{source_name}_missing_image'
    flagged_rows.append(flagged)
    df = df[~mask_missing]
    print(f"   🚩 Flagged: {mask_missing.sum():,}")
    
    # Issue 2: Invalid dimensions (only if columns exist)
    if has_bbox_coords:
        print(f"2️⃣ Checking for invalid box dimensions...")
        mask_invalid_dims = (
            (df['w'] == 9999) | (df['w'] == -1) | (df['w'] <= 0) |
            (df['h'] == 9999) | (df['h'] == -1) | (df['h'] <= 0) |
            df['w'].isna() | df['h'].isna()
        )
        flagged = df[mask_invalid_dims].copy()
        flagged['flag_reason'] = f'{source_name}_invalid_box_dimensions'
        flagged_rows.append(flagged)
        df = df[~mask_invalid_dims]
        print(f"   🚩 Flagged: {mask_invalid_dims.sum():,}")
        
        # Issue 3: Invalid coordinates
        print(f"3️⃣ Checking for invalid coordinates...")
        mask_invalid_coords = (df['x'] < 0) | (df['y'] < 0) | df['x'].isna() | df['y'].isna()
        flagged = df[mask_invalid_coords].copy()
        flagged['flag_reason'] = f'{source_name}_invalid_coordinates'
        flagged_rows.append(flagged)
        df = df[~mask_invalid_coords]
        print(f"   🚩 Flagged: {mask_invalid_coords.sum():,}")
        
        # Issue 4: Boxes outside image bounds
        if 'OriginalImage[Width' in meta_cols and 'OriginalImage[Height' in meta_cols:
            print(f"4️⃣ Checking if boxes fit inside images...")
            out_of_bounds = []
            valid_boxes = []
            
            for idx, row in df.iterrows():
                img_name = row['Image Index']
                if img_name in image_meta:
                    meta = image_meta[img_name]
                    if 'OriginalImage[Width' in meta and 'OriginalImage[Height' in meta:
                        img_width = meta['OriginalImage[Width']
                        img_height = meta['OriginalImage[Height']
                        
                        if (row['x'] + row['w'] > img_width) or (row['y'] + row['h'] > img_height):
                            out_of_bounds.append(idx)
                        else:
                            valid_boxes.append(idx)
                    else:
                        valid_boxes.append(idx)
                else:
                    out_of_bounds.append(idx)
            
            if out_of_bounds:
                flagged = df.loc[out_of_bounds].copy()
                flagged['flag_reason'] = f'{source_name}_box_out_of_bounds'
                flagged_rows.append(flagged)
            df = df.loc[valid_boxes] if valid_boxes else pd.DataFrame()
            print(f"   🚩 Flagged: {len(out_of_bounds):,}")
    else:
        print(f"⚠️  Skipping bbox coordinate validation (columns not found)")
    
    # Issue 5: Finding Label mismatch
    if 'Finding Label' in df.columns and 'Finding Labels' in meta_cols:
        print(f"5️⃣ Checking Finding Label consistency...")
        mismatch = []
        valid_labels = []
        
        for idx, row in df.iterrows():
            img_name = row['Image Index']
            if img_name in image_meta and 'Finding Labels' in image_meta[img_name]:
                img_labels = image_meta[img_name]['Finding Labels']
                box_label = row['Finding Label']
                
                if pd.notna(box_label) and pd.notna(img_labels):
                    if box_label not in str(img_labels):
                        mismatch.append(idx)
                    else:
                        valid_labels.append(idx)
                else:
                    valid_labels.append(idx)
            else:
                valid_labels.append(idx)
        
        if mismatch:
            flagged = df.loc[mismatch].copy()
            flagged['flag_reason'] = f'{source_name}_label_mismatch'
            flagged_rows.append(flagged)
        df = df.loc[valid_labels] if valid_labels else pd.DataFrame()
        print(f"   🚩 Flagged: {len(mismatch):,}")
    
    # Combine all flagged
    all_flagged = pd.concat(flagged_rows, ignore_index=True) if flagged_rows else pd.DataFrame()
    
    print(f"\n✅ CLEAN: {len(df):,} boxes ({len(df)/initial_count*100:.1f}%)")
    print(f"🚩 FLAGGED: {len(all_flagged):,} boxes ({len(all_flagged)/initial_count*100:.1f}%)")
    
    return df, all_flagged

In [ ]:
# Clean bbox_2.csv
print("=" * 70)
print("📦 PART B: CLEANING BOUNDING BOXES")
print("=" * 70)

print("\n" + "-" * 70)
print("Cleaning bbox_2.csv")
print("-" * 70)
bbox_2_clean, bbox_2_flagged = clean_bbox_table(bbox_2, data_entry_clean, 'bbox_2')

# Clean corrupted bbox
print("\n" + "-" * 70)
print("Cleaning HACKATHON_CORRUPTED_bbox_list.csv")
print("-" * 70)
bbox_corrupted_clean, bbox_corrupted_flagged = clean_bbox_table(bbox_corrupted, data_entry_clean, 'bbox_corrupted')

print("\n📊 Bounding Box Cleaning Summary:")
print(f"   bbox_2 clean: {len(bbox_2_clean):,}")
print(f"   bbox_2 flagged: {len(bbox_2_flagged):,}")
print(f"   bbox_corrupted clean: {len(bbox_corrupted_clean):,}")
print(f"   bbox_corrupted flagged: {len(bbox_corrupted_flagged):,}")

---
# 🔄 SECTION 8: Salvage Data from Corrupted Data Entry
---
Extract valid rows from the corrupted data_entry file that aren't already in the clean set.

In [ ]:
def salvage_corrupted_data_entry(corrupted_df, clean_df):
    """
    Try to salvage good rows from corrupted data_entry
    
    Args:
        corrupted_df: The corrupted data_entry DataFrame
        clean_df: The already cleaned data_entry DataFrame
        
    Returns:
        tuple: (salvaged_df, flagged_df)
    """
    df = corrupted_df.copy()
    initial_count = len(df)
    
    print(f"🔍 Starting with {initial_count:,} rows")
    
    # Get images already in clean set
    existing_images = set(clean_df['Image Index'].values)
    
    # Remove duplicates with clean set
    mask_new = ~df['Image Index'].isin(existing_images)
    df = df[mask_new]
    print(f"🔄 Removed {(~mask_new).sum():,} duplicates already in clean set")
    print(f"📝 Processing {len(df):,} unique rows")
    
    # Apply same cleaning rules
    df_salvaged, df_flagged_salvage = clean_data_entry_primary(df)
    
    return df_salvaged, df_flagged_salvage

# Salvage data from corrupted data_entry
print("=" * 70)
print("🔄 PART C: SALVAGING DATA FROM CORRUPTED DATA_ENTRY")
print("=" * 70)

data_entry_salvaged, data_entry_salvaged_flagged = salvage_corrupted_data_entry(
    data_entry_corrupted, 
    data_entry_clean
)

print(f"\n📊 Salvage Results:")
print(f"   ✅ SALVAGED: {len(data_entry_salvaged):,} additional clean rows")
print(f"   🚩 FLAGGED: {len(data_entry_salvaged_flagged):,} flagged rows")

---
# 🔗 SECTION 9: Combine Clean Data Entry with Bounding Boxes
---
Merge all clean data_entry sources with bounding box data using LEFT JOIN.

In [ ]:
print("=" * 70)
print("🔗 PART D: COMBINING CLEAN DATA_ENTRY + BBOX")
print("=" * 70)

# Combine all clean data_entry sources
all_clean_data_entry = pd.concat([data_entry_clean, data_entry_salvaged], ignore_index=True)
all_clean_data_entry = all_clean_data_entry.drop_duplicates(subset=['Image Index'], keep='first')

print(f"📊 Total clean data_entry records: {len(all_clean_data_entry):,}")

# Combine all clean bbox sources
all_clean_bbox = pd.concat([bbox_2_clean, bbox_corrupted_clean], ignore_index=True)

# Determine which columns to use for deduplication
dedup_cols = ['Image Index']
if 'Finding Label' in all_clean_bbox.columns:
    dedup_cols.append('Finding Label')
if 'x' in all_clean_bbox.columns:
    dedup_cols.append('x')
if 'y' in all_clean_bbox.columns:
    dedup_cols.append('y')

# Deduplicate
if len(dedup_cols) > 1:
    all_clean_bbox = all_clean_bbox.drop_duplicates(subset=dedup_cols, keep='first')
    print(f"📊 Deduplicated bbox using columns: {dedup_cols}")
else:
    all_clean_bbox = all_clean_bbox.drop_duplicates(subset=['Image Index'], keep='first')
    print(f"📊 Deduplicated bbox using only: Image Index")

print(f"📊 Total clean bbox records: {len(all_clean_bbox):,}")

# Merge data_entry with bbox (LEFT JOIN)
print(f"\n🔗 BBox columns available: {list(all_clean_bbox.columns)}")

if len(all_clean_bbox) > 0:
    clean_entry_combined = all_clean_data_entry.merge(
        all_clean_bbox,
        on='Image Index',
        how='left',
        suffixes=('', '_bbox')
    )
    
    # Clean up column names
    if 'Finding Label' in clean_entry_combined.columns:
        if 'Finding Labels' in clean_entry_combined.columns:
            clean_entry_combined = clean_entry_combined.rename(columns={
                'Finding Labels': 'diseases',
                'Finding Label': 'bbox_disease'
            })
else:
    clean_entry_combined = all_clean_data_entry.copy()
    print("⚠️  No bbox data to merge")

print(f"\n✅ CLEAN_ENTRY.CSV: {len(clean_entry_combined):,} records")
if 'bbox_disease' in clean_entry_combined.columns:
    print(f"   📦 With bounding boxes: {clean_entry_combined['bbox_disease'].notna().sum():,}")
    print(f"   📦 Without bounding boxes: {clean_entry_combined['bbox_disease'].isna().sum():,}")
else:
    print(f"   📦 No bounding box data available")

---
# 🚩 SECTION 10: Combine Flagged Data Entry with Bounding Boxes
---
Merge all flagged data sources using OUTER JOIN to capture all flagged records.

In [ ]:
print("=" * 70)
print("🚩 PART E: COMBINING FLAGGED DATA_ENTRY + BBOX")
print("=" * 70)

# Combine all flagged data_entry sources
all_flagged_data_entry = pd.concat([
    data_entry_flagged, 
    data_entry_salvaged_flagged
], ignore_index=True)
all_flagged_data_entry = all_flagged_data_entry.drop_duplicates(subset=['Image Index'], keep='first')

print(f"📊 Total flagged data_entry records: {len(all_flagged_data_entry):,}")

# Combine all flagged bbox sources
all_flagged_bbox = pd.concat([bbox_2_flagged, bbox_corrupted_flagged], ignore_index=True)

# Determine deduplication columns
dedup_cols_flagged = ['Image Index']
if 'Finding Label' in all_flagged_bbox.columns:
    dedup_cols_flagged.append('Finding Label')
if 'x' in all_flagged_bbox.columns:
    dedup_cols_flagged.append('x')
if 'y' in all_flagged_bbox.columns:
    dedup_cols_flagged.append('y')

# Deduplicate
if len(dedup_cols_flagged) > 1:
    all_flagged_bbox = all_flagged_bbox.drop_duplicates(subset=dedup_cols_flagged, keep='first')
else:
    all_flagged_bbox = all_flagged_bbox.drop_duplicates(subset=['Image Index'], keep='first')

print(f"📊 Total flagged bbox records: {len(all_flagged_bbox):,}")

# Merge flagged data_entry with flagged bbox (OUTER JOIN)
print(f"\n🔗 Flagged BBox columns available: {list(all_flagged_bbox.columns)}")

if len(all_flagged_bbox) > 0:
    flagged_combined = all_flagged_data_entry.merge(
        all_flagged_bbox,
        on='Image Index',
        how='outer',
        suffixes=('', '_bbox')
    )
    
    # Clean up column names
    if 'Finding Label' in flagged_combined.columns:
        if 'Finding Labels' in flagged_combined.columns:
            flagged_combined = flagged_combined.rename(columns={
                'Finding Labels': 'diseases',
                'Finding Label': 'bbox_disease'
            })
else:
    flagged_combined = all_flagged_data_entry.copy()
    print("⚠️  No flagged bbox data to merge")

print(f"\n🚩 FLAGGED.CSV: {len(flagged_combined):,} records")

---
# 💾 SECTION 11: Save Initial Cleaning Results
---
Save clean_entry.csv and flagged.csv, generate summary statistics, and copy to Google Drive.

In [ ]:
print("=" * 70)
print("💾 PART F: SAVING INITIAL CLEANING RESULTS")
print("=" * 70)

# Save clean_entry.csv
clean_entry_combined.to_csv('/content/cleaned_data/clean_entry.csv', index=False)
print(f"✅ Saved: clean_entry.csv ({len(clean_entry_combined):,} rows)")

# Save flagged.csv
flagged_combined.to_csv('/content/flagged_data/flagged.csv', index=False)
print(f"🚩 Saved: flagged.csv ({len(flagged_combined):,} rows)")

# Save summary statistics
summary = {
    'Total Original Records': len(data_entry) + len(data_entry_corrupted),
    'Clean Records': len(clean_entry_combined),
    'Flagged Records': len(flagged_combined),
    'Clean Percentage': f"{len(clean_entry_combined)/(len(data_entry) + len(data_entry_corrupted))*100:.2f}%",
}

# Add bbox stats
if 'bbox_disease' in clean_entry_combined.columns:
    summary['Clean with BBox'] = int(clean_entry_combined['bbox_disease'].notna().sum())
    summary['Clean without BBox'] = int(clean_entry_combined['bbox_disease'].isna().sum())
else:
    summary['Clean with BBox'] = 0
    summary['Clean without BBox'] = len(clean_entry_combined)

summary_df = pd.DataFrame([summary])
summary_df.to_csv('/content/logs/cleaning_summary.csv', index=False)
print(f"📊 Saved: cleaning_summary.csv")

# Save detailed flag reasons
if 'flag_reason' in flagged_combined.columns:
    flag_summary = flagged_combined['flag_reason'].value_counts().reset_index()
    flag_summary.columns = ['Flag Reason', 'Count']
    flag_summary.to_csv('/content/logs/flag_reasons.csv', index=False)
    print(f"📋 Saved: flag_reasons.csv")
    
    print("\n📋 Top 10 flag reasons:")
    print(flag_summary.head(10).to_string(index=False))

# Copy to Google Drive
drive_output = '/content/drive/MyDrive/xray_cleaned_dataset'
os.makedirs(drive_output, exist_ok=True)

shutil.copy('/content/cleaned_data/clean_entry.csv', f'{drive_output}/clean_entry.csv')
shutil.copy('/content/flagged_data/flagged.csv', f'{drive_output}/flagged.csv')
shutil.copytree('/content/logs', f'{drive_output}/logs', dirs_exist_ok=True)

print(f"\n✅ All files saved to Google Drive: {drive_output}")

---
# 🎉 PHASE 1 COMPLETE: Initial Cleaning Summary
---
Initial cleaning is complete! We now have:
- **clean_entry.csv**: Clean data with bounding boxes
- **flagged.csv**: Data with quality issues for further processing

Now we proceed to **PHASE 2: Full Cleaning** to further refine the flagged data.

---

In [ ]:
# Initial Cleaning Summary
print("=" * 70)
print("🎉 PHASE 1 COMPLETE: INITIAL CLEANING")
print("=" * 70)

print(f"""
📊 INITIAL CLEANING STATISTICS:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

✅ CLEAN_ENTRY.CSV
   Total records: {len(clean_entry_combined):,}
   With bounding boxes: {clean_entry_combined['bbox_disease'].notna().sum() if 'bbox_disease' in clean_entry_combined.columns else 0:,}
   Without bounding boxes: {clean_entry_combined['bbox_disease'].isna().sum() if 'bbox_disease' in clean_entry_combined.columns else len(clean_entry_combined):,}

🚩 FLAGGED.CSV
   Total records: {len(flagged_combined):,}
   
📈 OVERALL
   Clean rate: {len(clean_entry_combined)/(len(data_entry) + len(data_entry_corrupted))*100:.1f}%
   
📁 SAVED TO: {drive_output}
   ├── clean_entry.csv       (Ready for ML!)
   ├── flagged.csv           (For further cleaning)
   └── logs/
       ├── cleaning_summary.csv
       └── flag_reasons.csv

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
""")

---
# 🔄 PHASE 2: FULL CLEANING PIPELINE
================================================================================

This phase performs a comprehensive 5-stage data cleaning process on the initial results.

**STAGES:**
1. Load and assess initial data (clean_entry.csv + flagged.csv)
2. Analyze flagged data quality issues
3. Clean flagged data → cleaned2.0.csv
4. Extract completely clean data → cleaned3.0.csv
5. Merge all clean data → final_clean.csv

---

---
# 📂 SECTION 12: Load Initial Cleaned Data for Full Cleaning
---
Load clean_entry.csv and flagged.csv from the initial cleaning stage.

In [ ]:
# Configuration for Full Cleaning
INPUT_FILES = {
    'clean_entry': '/content/cleaned_data/clean_entry.csv',
    'flagged': '/content/flagged_data/flagged.csv'
}

OUTPUT_FILES = {
    'cleaned2': '/content/cleaned_data/cleaned2.0.csv',
    'cleaned3': '/content/cleaned_data/cleaned3.0.csv',
    'final': '/content/cleaned_data/final_clean.csv',
    'report': '/content/logs/cleaning_report.txt'
}

KEY_COLUMNS = [
    'Patient Gender', 
    'Patient ID', 
    'Image Index', 
    'Patient Age', 
    'diseases', 
    'OriginalImage[Width', 
    'OriginalImage[Height', 
    'View Position', 
    'Random_Code'
]

# Age validation parameters
MIN_VALID_AGE = 0
MAX_VALID_AGE = 120
UNREALISTIC_AGES = [999, 150]

print("✅ Configuration set for Full Cleaning Pipeline")

In [ ]:
# Helper functions for formatted output
def print_stage(stage_num, title):
    """Print formatted stage header"""
    print("\n" + "=" * 80)
    print(f"STAGE {stage_num}: {title}")
    print("=" * 80)

def print_section(title):
    """Print formatted section header"""
    print("\n" + "-" * 80)
    print(f"  {title}")
    print("-" * 80)

# STAGE 1: Load and Assess Initial Data
print_stage(1, "LOAD AND ASSESS INITIAL DATA")

# Load clean_entry.csv
print_section("Loading clean_entry.csv")
df_clean_entry = pd.read_csv(INPUT_FILES['clean_entry'])
print(f"✅ Loaded: {len(df_clean_entry):,} rows")
print(f"   Columns: {len(df_clean_entry.columns)}")
print(f"   Unique images: {df_clean_entry['Image Index'].nunique():,}")
print(f"   Unique patients: {df_clean_entry['Patient ID'].nunique():,}")

# Load flagged.csv
print_section("Loading flagged.csv")
df_flagged = pd.read_csv(INPUT_FILES['flagged'])
print(f"✅ Loaded: {len(df_flagged):,} rows")
print(f"   Columns: {len(df_flagged.columns)}")

# Analyze flag reasons
if 'flag_reason' in df_flagged.columns:
    print("\n📋 Flag reasons breakdown:")
    for reason, count in df_flagged['flag_reason'].value_counts().items():
        print(f"   - {reason}: {count:,} rows")

---
# 🔍 SECTION 13: Analyze Flagged Data Quality Issues
---
Analyze gender values, age issues, and missing data in flagged.csv.

In [ ]:
# STAGE 2: Analyze Flagged Data Quality Issues
print_stage(2, "ANALYZE FLAGGED DATA QUALITY ISSUES")

# Gender analysis
print_section("Gender Value Analysis")
print("📋 Unique gender values:")
for gender, count in df_flagged['Patient Gender'].value_counts(dropna=False).items():
    print(f"   - '{gender}': {count:,} rows")

# Age analysis
print_section("Age Value Analysis")
print("📋 Sample of problematic age values:")
age_counts = df_flagged['Patient Age'].value_counts().head(20)
for age, count in age_counts.items():
    print(f"   - '{age}': {count:,} rows")

# Missing data analysis
print_section("Missing Data Analysis")
print("📋 NaN counts in key columns:")
for col in KEY_COLUMNS:
    if col in df_flagged.columns:
        nan_count = df_flagged[col].isna().sum()
        pct = (nan_count / len(df_flagged)) * 100
        print(f"   - {col:30s}: {nan_count:5,} ({pct:5.1f}%)")

---
# 🧼 SECTION 14: Clean Gender Column
---
Standardize gender values: 'female'/'f' → 'F', 'male'/'m' → 'M', all others → NaN.

In [ ]:
def clean_gender(gender):
    """
    Standardize gender values to F, M, or NaN
    
    Args:
        gender: Original gender value
        
    Returns:
        Standardized gender ('F', 'M', or NaN)
    """
    if pd.isna(gender):
        return np.nan
    
    gender_str = str(gender).strip().lower()
    
    # Map to standard F/M
    if gender_str in ['f', 'female']:
        return 'F'
    elif gender_str in ['m', 'male']:
        return 'M'
    else:
        # Keep unknown, N, or other invalid values as NaN
        return np.nan

# STAGE 3 (Part 1): Clean Gender
print_stage(3, "CLEAN FLAGGED DATA → cleaned2.0.csv")

df_cleaned2 = df_flagged.copy()
initial_rows = len(df_cleaned2)

print_section("Cleaning Gender Column")
original_valid_gender = df_cleaned2['Patient Gender'].notna().sum()
df_cleaned2['Patient Gender'] = df_cleaned2['Patient Gender'].apply(clean_gender)
cleaned_valid_gender = df_cleaned2['Patient Gender'].notna().sum()

print(f"  Before: {original_valid_gender:,} valid gender values")
print(f"  After:  {cleaned_valid_gender:,} valid gender values (F/M only)")
print(f"  Improvement: +{cleaned_valid_gender - original_valid_gender:,} standardized values")

---
# 🔢 SECTION 15: Clean Age Column
---
Convert and validate ages: handle text ages, round decimals, filter unrealistic values.

In [ ]:
def clean_age(age):
    """
    Convert and validate age values
    
    Args:
        age: Original age value
        
    Returns:
        Cleaned age (integer 0-120, or NaN if invalid)
    """
    if pd.isna(age):
        return np.nan
    
    # If already numeric and valid, return as float
    try:
        age_num = float(age)
        # Filter out unrealistic ages
        if age_num < MIN_VALID_AGE or age_num > MAX_VALID_AGE or age_num in UNREALISTIC_AGES:
            return np.nan
        # Handle decimal ages - round to nearest integer
        return round(age_num)
    except (ValueError, TypeError):
        pass
    
    # Handle text ages
    age_str = str(age).strip().lower()
    
    # Map common text ages
    text_age_map = {
        'twenty five': 25,
        'unknown': np.nan,
    }
    
    if age_str in text_age_map:
        return text_age_map[age_str]
    
    # If we can't convert, return NaN
    return np.nan

# STAGE 3 (Part 2): Clean Age
print_section("Cleaning Age Column")
original_valid_age = df_cleaned2['Patient Age'].notna().sum()
df_cleaned2['Patient Age'] = df_cleaned2['Patient Age'].apply(clean_age)
cleaned_valid_age = df_cleaned2['Patient Age'].notna().sum()

print(f"  Before: {original_valid_age:,} non-null age values")
print(f"  After:  {cleaned_valid_age:,} valid age values (0-120)")

if cleaned_valid_age > 0:
    print(f"  Age range: {df_cleaned2['Patient Age'].min():.0f} to {df_cleaned2['Patient Age'].max():.0f}")
    print(f"  Mean age: {df_cleaned2['Patient Age'].mean():.1f} years")

---
# 💾 SECTION 16: Save Cleaned Flagged Data (cleaned2.0.csv)
---
Save the cleaned flagged data with standardized gender and age values.

In [ ]:
# STAGE 3 (Part 3): Save cleaned2.0.csv
print_section("Cleaning Summary")
print(f"  Total rows retained: {len(df_cleaned2):,} (100% - no rows removed)")
print(f"  Gender standardized: {df_cleaned2['Patient Gender'].notna().sum():,} valid (F/M)")
print(f"  Age cleaned: {df_cleaned2['Patient Age'].notna().sum():,} valid (0-120)")

# Save cleaned2.0.csv
df_cleaned2.to_csv(OUTPUT_FILES['cleaned2'], index=False)
print(f"\n✅ Saved: {OUTPUT_FILES['cleaned2']}")

---
# ✨ SECTION 17: Extract Completely Clean Data (cleaned3.0.csv)
---
Filter to keep only rows with NO NaN values in key columns.

In [ ]:
# STAGE 4: Extract Completely Clean Data
print_stage(4, "EXTRACT COMPLETELY CLEAN DATA → cleaned3.0.csv")

# Check NaN counts
print_section("NaN Analysis in Key Columns")
for col in KEY_COLUMNS:
    if col in df_cleaned2.columns:
        nan_count = df_cleaned2[col].isna().sum()
        pct = (nan_count / len(df_cleaned2)) * 100
        print(f"  {col:30s}: {nan_count:5,} NaN ({pct:5.1f}%)")

# Create filter mask for completely clean data
print_section("Filtering for Complete Data")

# Check which key columns exist
existing_key_cols = [col for col in KEY_COLUMNS if col in df_cleaned2.columns]
clean_mask = df_cleaned2[existing_key_cols].notna().all(axis=1)

# Also check that diseases is not empty string
if 'diseases' in df_cleaned2.columns:
    clean_mask &= df_cleaned2['diseases'].str.strip() != ''

df_cleaned3 = df_cleaned2[clean_mask].copy()

rows_removed = len(df_cleaned2) - len(df_cleaned3)
pct_clean = (len(df_cleaned3) / len(df_cleaned2)) * 100 if len(df_cleaned2) > 0 else 0

print(f"  Total rows: {len(df_cleaned2):,}")
print(f"  Complete rows: {len(df_cleaned3):,}")
print(f"  Rows removed: {rows_removed:,}")
print(f"  Clean data percentage: {pct_clean:.1f}%")

In [ ]:
# Display clean data statistics
print_section("Clean Data Statistics")

if len(df_cleaned3) > 0:
    print(f"📊 Gender distribution:")
    for gender, count in df_cleaned3['Patient Gender'].value_counts().items():
        print(f"   - {gender}: {count:,}")
    
    print(f"\n📊 Age statistics:")
    print(f"   - Count: {len(df_cleaned3):,}")
    print(f"   - Min: {df_cleaned3['Patient Age'].min():.0f}")
    print(f"   - Max: {df_cleaned3['Patient Age'].max():.0f}")
    print(f"   - Mean: {df_cleaned3['Patient Age'].mean():.1f}")
    
    if 'View Position' in df_cleaned3.columns:
        print(f"\n📊 View Position distribution:")
        for view, count in df_cleaned3['View Position'].value_counts().items():
            print(f"   - {view}: {count:,}")
    
    if 'diseases' in df_cleaned3.columns:
        print(f"\n📊 Top 5 diseases:")
        for disease, count in df_cleaned3['diseases'].value_counts().head(5).items():
            print(f"   - {disease}: {count:,}")
else:
    print("⚠️  No completely clean rows found after filtering")

# Save cleaned3.0.csv
df_cleaned3.to_csv(OUTPUT_FILES['cleaned3'], index=False)
print(f"\n✅ Saved: {OUTPUT_FILES['cleaned3']}")

---
# 🔀 SECTION 18: Merge All Clean Data (final_clean.csv)
---
Combine clean_entry.csv and cleaned3.0.csv, remove duplicates, create final dataset.

In [ ]:
# STAGE 5: Merge All Clean Data
print_stage(5, "MERGE ALL CLEAN DATA → final_clean.csv")

# Check column compatibility
print_section("Checking Column Compatibility")
cols1 = set(df_clean_entry.columns)
cols2 = set(df_cleaned3.columns)

print(f"  clean_entry.csv columns: {len(cols1)}")
print(f"  cleaned3.0.csv columns: {len(cols2)}")

# Align columns if different
if cols1 != cols2:
    print("\n  Column differences found - aligning...")
    only_in_clean_entry = cols1 - cols2
    only_in_cleaned3 = cols2 - cols1
    
    if only_in_clean_entry:
        print(f"    Only in clean_entry: {only_in_clean_entry}")
    if only_in_cleaned3:
        print(f"    Only in cleaned3.0: {only_in_cleaned3}")
    
    # Add missing columns
    all_cols = sorted(cols1.union(cols2))
    for col in all_cols:
        if col not in df_clean_entry.columns:
            df_clean_entry[col] = None
        if col not in df_cleaned3.columns:
            df_cleaned3[col] = None
    
    # Reorder columns
    df_clean_entry = df_clean_entry[all_cols]
    df_cleaned3 = df_cleaned3[all_cols]
    print("    ✅ Columns aligned!")

# Combine datasets
print_section("Combining Datasets")
print(f"  clean_entry.csv: {len(df_clean_entry):,} rows")
print(f"  cleaned3.0.csv:  {len(df_cleaned3):,} rows")

df_final = pd.concat([df_clean_entry, df_cleaned3], ignore_index=True)
print(f"  Total after concat: {len(df_final):,} rows")

In [ ]:
# Remove duplicates
print_section("Removing Duplicates")
initial_count = len(df_final)
df_final = df_final.drop_duplicates(subset=['Image Index'], keep='first')
duplicates_removed = initial_count - len(df_final)

print(f"  Before deduplication: {initial_count:,} rows")
print(f"  After deduplication:  {len(df_final):,} rows")
print(f"  Duplicates removed:   {duplicates_removed:,} rows")

# Verify uniqueness
is_unique = df_final['Image Index'].nunique() == len(df_final)
print(f"  All images unique: {'✅ YES' if is_unique else '❌ NO'}")

# Final statistics
print_section("Final Dataset Statistics")
print(f"  Total rows: {len(df_final):,}")
print(f"  Unique images: {df_final['Image Index'].nunique():,}")
print(f"  Unique patients: {df_final['Patient ID'].nunique():,}")

print(f"\n📊 Gender distribution:")
for gender, count in df_final['Patient Gender'].value_counts(dropna=False).items():
    print(f"   - {gender}: {count:,}")

print(f"\n📊 Age statistics:")
valid_ages = df_final['Patient Age'].notna().sum()
print(f"   - Valid ages: {valid_ages:,}")
print(f"   - Missing ages: {df_final['Patient Age'].isna().sum():,}")
if valid_ages > 0:
    print(f"   - Min: {df_final['Patient Age'].min():.0f}")
    print(f"   - Max: {df_final['Patient Age'].max():.0f}")
    print(f"   - Mean: {df_final['Patient Age'].mean():.1f}")

if 'View Position' in df_final.columns:
    print(f"\n📊 View Position distribution:")
    for view, count in df_final['View Position'].value_counts(dropna=False).items():
        print(f"   - {view}: {count:,}")

# Determine diseases column name
diseases_col = 'diseases' if 'diseases' in df_final.columns else 'Finding Labels'
if diseases_col in df_final.columns:
    print(f"\n📊 Top 10 diseases:")
    for disease, count in df_final[diseases_col].value_counts().head(10).items():
        print(f"   - {disease}: {count:,}")

# Save final_clean.csv
df_final.to_csv(OUTPUT_FILES['final'], index=False)
print(f"\n✅ Saved: {OUTPUT_FILES['final']}")

---
# 📝 SECTION 19: Generate Final Cleaning Report
---
Generate comprehensive cleaning_report.txt with statistics for all stages.

In [ ]:
# Generate Cleaning Report
print_stage("FINAL", "GENERATING CLEANING REPORT")

report_lines = []
report_lines.append("=" * 80)
report_lines.append("MEDICAL IMAGING DATASET - COMPLETE CLEANING REPORT")
report_lines.append("=" * 80)
report_lines.append(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
report_lines.append("")

# Stage 1: Input Data
report_lines.append("STAGE 1: INPUT DATA")
report_lines.append("-" * 80)
report_lines.append(f"clean_entry.csv:  {len(df_clean_entry):,} rows (pre-cleaned data)")
report_lines.append(f"flagged.csv:      {len(df_flagged):,} rows (data with issues)")
report_lines.append(f"Total input:      {len(df_clean_entry) + len(df_flagged):,} rows")
report_lines.append("")

# Stage 2: Flag Analysis
report_lines.append("STAGE 2: FLAGGED DATA ISSUES")
report_lines.append("-" * 80)
if 'flag_reason' in df_flagged.columns:
    for reason, count in df_flagged['flag_reason'].value_counts().items():
        report_lines.append(f"  {reason}: {count:,} rows")
report_lines.append("")

# Stage 3: Cleaned Flagged Data
report_lines.append("STAGE 3: CLEANED FLAGGED DATA (cleaned2.0.csv)")
report_lines.append("-" * 80)
report_lines.append(f"Total rows: {len(df_cleaned2):,} (100% retained)")
report_lines.append(f"Valid gender (F/M): {df_cleaned2['Patient Gender'].notna().sum():,}")
report_lines.append(f"Valid age (0-120): {df_cleaned2['Patient Age'].notna().sum():,}")
report_lines.append("")

# Stage 4: Completely Clean Data
report_lines.append("STAGE 4: COMPLETELY CLEAN DATA (cleaned3.0.csv)")
report_lines.append("-" * 80)
report_lines.append(f"Total rows: {len(df_cleaned3):,}")
if len(df_flagged) > 0:
    report_lines.append(f"Percentage of flagged data: {len(df_cleaned3)/len(df_flagged)*100:.1f}%")
report_lines.append(f"All key columns: 100% complete (no NaN)")
report_lines.append("")

# Stage 5: Final Merged Data
report_lines.append("STAGE 5: FINAL MERGED DATA (final_clean.csv)")
report_lines.append("-" * 80)
report_lines.append(f"Total unique rows: {len(df_final):,}")
report_lines.append(f"Unique images: {df_final['Image Index'].nunique():,}")
report_lines.append(f"Unique patients: {df_final['Patient ID'].nunique():,}")
report_lines.append(f"Duplicates removed: {(len(df_clean_entry) + len(df_cleaned3)) - len(df_final):,}")
report_lines.append("")

# Final Quality Metrics
report_lines.append("FINAL QUALITY METRICS")
report_lines.append("-" * 80)
report_lines.append(f"Gender completeness: {df_final['Patient Gender'].notna().sum()/len(df_final)*100:.1f}%")
report_lines.append(f"Age completeness: {df_final['Patient Age'].notna().sum()/len(df_final)*100:.1f}%")

diseases_col = 'diseases' if 'diseases' in df_final.columns else 'Finding Labels'
if diseases_col in df_final.columns:
    report_lines.append(f"Disease labels: {df_final[diseases_col].notna().sum()/len(df_final)*100:.1f}%")
report_lines.append("")

report_lines.append("Gender Distribution:")
for gender, count in df_final['Patient Gender'].value_counts(dropna=False).items():
    pct = count / len(df_final) * 100
    report_lines.append(f"  {gender}: {count:,} ({pct:.1f}%)")
report_lines.append("")

valid_ages = df_final['Patient Age'].notna().sum()
if valid_ages > 0:
    report_lines.append("Age Statistics:")
    report_lines.append(f"  Valid: {valid_ages:,}")
    report_lines.append(f"  Min: {df_final['Patient Age'].min():.0f}")
    report_lines.append(f"  Max: {df_final['Patient Age'].max():.0f}")
    report_lines.append(f"  Mean: {df_final['Patient Age'].mean():.1f}")
    report_lines.append("")

if diseases_col in df_final.columns:
    report_lines.append("Top 10 Diseases:")
    for i, (disease, count) in enumerate(df_final[diseases_col].value_counts().head(10).items(), 1):
        pct = count / len(df_final) * 100
        report_lines.append(f"  {i:2d}. {disease}: {count:,} ({pct:.1f}%)")
    report_lines.append("")

# Output files
report_lines.append("OUTPUT FILES GENERATED")
report_lines.append("-" * 80)
for key, filename in OUTPUT_FILES.items():
    if os.path.exists(filename):
        size = os.path.getsize(filename) / (1024 * 1024)  # MB
        report_lines.append(f"  ✓ {filename} ({size:.2f} MB)")
report_lines.append("")

report_lines.append("=" * 80)
report_lines.append("CLEANING COMPLETE - DATASET READY FOR ANALYSIS!")
report_lines.append("=" * 80)

# Write report
report_text = "\n".join(report_lines)
with open(OUTPUT_FILES['report'], 'w') as f:
    f.write(report_text)

print(report_text)
print(f"\n✅ Report saved: {OUTPUT_FILES['report']}")

---
# 💾 SECTION 20: Save All Files to Google Drive
---
Copy all output files to Google Drive and display final summary.

In [ ]:
# Save all files to Google Drive
print("=" * 70)
print("💾 SAVING ALL FILES TO GOOGLE DRIVE")
print("=" * 70)

# Copy cleaned files to Drive
shutil.copy(OUTPUT_FILES['cleaned2'], f'{drive_output}/cleaned2.0.csv')
print(f"✅ Copied: cleaned2.0.csv")

shutil.copy(OUTPUT_FILES['cleaned3'], f'{drive_output}/cleaned3.0.csv')
print(f"✅ Copied: cleaned3.0.csv")

shutil.copy(OUTPUT_FILES['final'], f'{drive_output}/final_clean.csv')
print(f"✅ Copied: final_clean.csv")

shutil.copy(OUTPUT_FILES['report'], f'{drive_output}/logs/cleaning_report.txt')
print(f"✅ Copied: cleaning_report.txt")

print(f"\n📁 All files saved to: {drive_output}")

---
# 🎉 PIPELINE COMPLETE!
---

In [ ]:
# Final Summary
print("\n" + "=" * 80)
print("🎉 COMPLETE DATA CLEANING PIPELINE - SUCCESS!")
print("=" * 80)
print(f"Completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Calculate file sizes
def get_file_size_mb(filepath):
    if os.path.exists(filepath):
        return os.path.getsize(filepath) / (1024 * 1024)
    return 0

diseases_col = 'diseases' if 'diseases' in df_final.columns else 'Finding Labels'

print(f"""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

📊 FINAL DATASET STATISTICS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

🎯 FINAL_CLEAN.CSV
   Total records:     {len(df_final):,}
   Unique images:     {df_final['Image Index'].nunique():,}
   Unique patients:   {df_final['Patient ID'].nunique():,}
   
📈 DATA QUALITY
   Gender complete:   {df_final['Patient Gender'].notna().sum()/len(df_final)*100:.1f}%
   Age complete:      {df_final['Patient Age'].notna().sum()/len(df_final)*100:.1f}%
   Disease labels:    {df_final[diseases_col].notna().sum()/len(df_final)*100:.1f}%

📁 OUTPUT FILES
   {drive_output}/
   ├── clean_entry.csv      ({get_file_size_mb(INPUT_FILES['clean_entry']):.2f} MB)
   ├── flagged.csv          ({get_file_size_mb(INPUT_FILES['flagged']):.2f} MB)
   ├── cleaned2.0.csv       ({get_file_size_mb(OUTPUT_FILES['cleaned2']):.2f} MB)
   ├── cleaned3.0.csv       ({get_file_size_mb(OUTPUT_FILES['cleaned3']):.2f} MB)
   ├── final_clean.csv      ({get_file_size_mb(OUTPUT_FILES['final']):.2f} MB) ⭐ READY FOR ML!
   └── logs/
       ├── cleaning_summary.csv
       ├── flag_reasons.csv
       └── cleaning_report.txt

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✅ Dataset is now ready for Machine Learning and Analysis!
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
""")